In [74]:
import pandas as pd
import random
import numpy as np
import cvxpy as cvx

#### Load Data

In [3]:
dfTripData = pd.read_csv("location_data.csv")


#### Script Parameters

In [4]:
intNumHouseNodes = 5
intNumCommNodes = 2
intAvgNumCarsNode = 5 # goal is to have about 5 * 5 total cars
intSdNumCarsNode = 1
intProfiles = dfTripData.shape[0]

# could make battery parameters a randomized instantiation
fltCarEff = 3.95 # mi/kWh
fltBatteryCap = 40. # kWh
fltChargeRate = 1.9 # kW
fltDt = 0.25 # delta T
fltHomeRate = 1.4 # kW
fltWorkRate = 7 # kW

#### Functions

In [5]:
# create residential nodes

def createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData):
    #initialize data frame to hold master
    dfNodesTrips = pd.DataFrame(columns=['TripId','ResNode','CommNode'])  

    for intNode in range(intNumHouseNodes):

        # find how many vehicles will be at that residential node
        intNodeNumCars = int(random.gauss(intAvgNumCarsNode,intSdNumCarsNode))
        # assign cars to that node
        lsCarIds = [] # container to hold trip id for that node
        lsCommNode = [] # container to hold where the commerical node 
        lsResNode = [] # container to hold the res node
        for intCar in range(intNodeNumCars):

            # find a car from the data frame
            ind = random.randint(0,intProfiles)
            lsCarIds.append(dfTripData['ID'].iloc[ind]) # add to car id container

            # save res node
            lsResNode.append(intNode)

            # find which commercial node that car goes to
            lsCommNode.append(random.randint(1,intNumCommNodes))

        # create data frame and add to master
        dfNode = pd.DataFrame(list(zip(lsCarIds,lsResNode,lsCommNode)), \
                              columns=['TripId','ResNode','CommNode'])

        dfNodesTrips = dfNodesTrips.append(dfNode)

    dfNodesTrips.reset_index(inplace=True, drop=True)
    
    return dfNodesTrips

    

In [6]:
def getSOCconstraints(dfNodesTrips, fltCarEff, fltChargeRate, fltDt):

    intNumProfs = dfNodesTrips.shape[0]
    arrSOCrequirements = np.zeros((intNumProfs, 96))

    for ind, strId in enumerate(dfNodesTrips['TripId']):

        # find miles travelled
        fltMiles = dfTripData.loc[dfTripData['ID'] == strId,'DIST_M'].values[0]

        # get a list of locations
        lsLocations = dfTripData.loc[dfTripData['ID'] == strId].values.flatten()[3:-1].tolist()

        # so that we can find how many intervals we are away
        intAway = sum(1 if strLoc == "A" else 0 for strLoc in lsLocations)

        # and then find the average miles travelled each interval
        fltAvgMileInterval = fltMiles / intAway

        intStart = 0 
        while intStart < 96 and 'A's in lsLocations[intStart:]:

            # Find the first first time we start moving
            intStart = intStart + lsLocations[intStart:].index('A') 

            # from there find when we arrive at a destination
            if 'W' in lsLocations[intStart:] and 'H' in lsLocations[intStart:]:
                intEnd = min(lsLocations[intStart:].index('H'), lsLocations[intStart:].index('W'))
            elif 'W' in lsLocations[intStart:]:
                intEnd = lsLocations[intStart:].index('W')
            elif 'H' in lsLocations[intStart:]:
                intEnd = lsLocations[intStart:].index('H')
            else: # we are going to assume they are driving the rest of the time??
                break
               

            # intervals driving for trip is then (intStart+intEnd +1) - intStart = intEnd =1
            # then we can find miles travelled for first trip 
            fltMilesTraveled = fltAvgMileInterval * (intEnd)
            fltKwhRequirement = fltMilesTraveled / fltCarEff # miles / (miles/kWh)

            # now find SOC required
            fltInitialSoc = fltKwhRequirement

            # so we need to to have the initial SOC at the interval before this time:
            arrSOCrequirements[ind,intStart-1] = fltInitialSoc
            
            # now let's back fill in SOC from there
            intNext = 2
            fltPreviousSOC = fltInitialSoc - fltChargeRate * fltDt
            while fltPreviousSOC > 0:
                
                # fill in the SOC needed in the previous time series
                arrSOCrequirements[ind, intStart - intNext] = fltPreviousSOC
                
                intNext += 1 # for next iteration go to the next previous time step
                fltPreviousSOC = fltPreviousSOC - fltChargeRate * fltDt
           

            intStart += intEnd # increment the starting point for finding the next trip

    return arrSOCrequirements


In [35]:
def getChargeRateConstraints(dfNodesTrips, dfTripData):
    '''
    Function to find the maximum charging rate at each time interval and whether it is
    available to charge. 
    '''

    intNumProfs = dfNodesTrips.shape[0]
    # this will hold the charging rate in kW
    arrChargeRate = np.zeros((intNumProfs, 96)) 
    # this will hold the binary of whether the vehicle can charge
    arrCanCharge = np.zeros((intNumProfs, 96))
    

    for ind, strId in enumerate(dfNodesTrips['TripId']):

        # get location sequence for that trip
        lsLocations = dfTripData.loc[dfTripData['ID'] == strId].iloc[0,3:-1].values

        # iterate throught the locations to get its charge rate and availability 
        for intInterval, strLoc in enumerate(lsLocations):

            if strLoc == 'H': # if at home max charge 
                arrChargeRate[ind, intInterval] = fltHomeRate
                arrCanCharge[ind,intInterval] = 1
            elif strLoc == 'W': # working charge rate
                arrChargeRate[ind, intInterval] = fltWorkRate
                arrChargeRate[ind,intInterval] = 1
            else: # you are moving and can't charge
                arrChargeRate[ind, intInterval] = 0
                arrChargeRate[ind, intInterval] = 0
    
    return arrChargeRate, arrCanCharge

In [64]:
def getInitialSoc(arrSOCconstraint): 
    '''
    Function to get an random initialization of the vehicle's SOC
    '''
    
    intNumVehicles = np.shape(arrSOCconstraint)[0]
    lsInitialSoc = np.zeros((intNumVehicles)) # holder to have the initial SOC of vehicles

    for intVehicle in range(intNumVehicles): # for each vehicle 

        fltSocStart = random.random() * fltBatteryCap
        
        # make sure that the random SOC is > the minimum SOC required
        while fltSocStart < arrSOCconstraint[intVehicle,0]:
            fltSocStart = random.random() * fltBatteryCap
        
        lsInitialSoc[intVehicle] = fltSocStart
            
    arrSOCconstraint[:,0] = lsInitialSoc
    
    return arrSOCconstraint

#### Execute

In [66]:
# call functions to get Nodes Dataframe
dfNodesTrips = createCarsNodes(intNumHouseNodes, intNumCommNodes, intAvgNumCarsNode, intSdNumCarsNode, \
                    intProfiles, dfTripData)

# now call function to get the SOC constraints array
arrSOCconstraint = getSOCconstraints(dfNodesTrips, fltCarEff, fltChargeRate, fltDt)

# call function to get an array of maximum charging rates
arrChargeRate, arrCanCharge = getChargeRateConstraints(dfNodesTrips, dfTripData)

# call function to get initial SOCs of vehicles
# replaces first column of soc constraint array
arrSOCconstraint = getInitialSoc(arrSOCconstraint)

#### Define OptimizationFunctions

In [175]:
def make_constraints(arrSOCconstraint, arrChargeRate, arrCanCharge, fltDt, fltBatteryCap):
    
    intVehicles = np.shape(arrSOCconstraint)[0]
    intTimes = np.shape(arrSOCconstraint)[1]
    
    # make variables
    varCharge = cvx.Variable((intVehicles,intTimes)) # charging rate of vehicles
    varDischarge = cvx.Variable((intVehicles,intTimes)) # discharging rate of vehicles
    varSOCs = cvx.Variable((intVehicles,intTimes)) # SOCs of the vehicles
    
    # initialzie constraints
    constraints = []
    
    # define the charge rate rate constraints
    constraints += [varCharge >= 0] # positively defined
    constraints += [varDischarge <= 0] # negatively defined
    constraints += [varSOCs >= 0] # can't have negative SOC
    constraints += [varSOCs <= fltBatteryCap] # can't over charge
    
    # define charging limits for each vehicle at each time
    for v in range(intVehicles):
    
        # loop through each time to model charging and SOC constraints
        for t in range(intTimes):
            
            # can only charge if present and max rate is given by array
            constraints += [varCharge[v, t] <= arrChargeRate[v,t]*arrCanCharge[v, t]]
            constraints += [-1*arrChargeRate[v,t]*arrCanCharge[v, t] <= varDischarge[v,t]]
            
            # vehicle's SOC must be greater than the minimum requirement
            constraints += [varSOCs[v,t] >= arrSOCconstraint[v,t]]
            
        
        for t in range(1,intTimes): # loop through from time 1 to end to set SOC
            
            constraints += [varSOCs[v,t]  == \
                           varSOCs[v,t-1] + varCharge[v,t]*fltDt + varDischarge[v,t]*fltDt]
        
        
    
    return constraints, varCharge, varDischarge, varSOCs
       
    

In [176]:
def make_objectives(constraints, varCharge, varDischarge, varSocs, arrCanCharge, arrChargeRate,\
                    arrSOCconstraint,fltBatteryCap,fltDt, fltRegDownVal, fltRegUpVal,lsElectric):
    
    # define regulation up and down variables
    varRegUp = cvx.Variable((np.shape(arrCanCharge)))
    varRegDown = cvx.Variable((np.shape(arrCanCharge)))
    
    # always positive
    constraints += [varRegUp >= 0]
    constraints += [varRegDown >= 0]
    
    intVehicles = np.shape(arrSOCconstraint)[0]
    intTimes = np.shape(arrSOCconstraint)[1]
    
    for v in range(intVehicles):
        for t in range(intTimes):
            
            # reg down constraints
            # it is at most the charging rate if it's available
            constraints += [varRegDown[v,t] <= arrCanCharge[v,t]*arrChargeRate[v,t]]
            # or what is left to charge batteries capacity
            constraints += [varRegDown[v,t] <= (fltBatteryCap - varSocs[v,t])/fltDt]
            
            # reg up constraints 
            # it is at most the charge rate if it's avalable 
            constraints += [varRegUp[v,t] <= arrCanCharge[v,t]*arrChargeRate[v,t]]
            # or what it's minimum SOC requriement is
            constraints += [varRegUp[v,t] <= (varSocs[v,t] - arrSOCconstraint[v,t])/fltDt]
            
    
    obj_value = cvx.sum(varRegDown - varCharge)*fltRegDownVal + \
                cvx.sum(varRegUp - varDischarge)*fltRegUpVal + \
                cvx.sum(lsElectric*cvx.sum(varCharge,axis=0)) # add battery degradation here
    
    return constraints, obj_value

In [177]:
fltRegDownVal=4
fltRegUpVal= 5
lsElectric = [0 for i in range(96)]

In [178]:
constraints, varCharge, varDischarge, varSocs = make_constraints(arrSOCconstraint,\
                                        arrChargeRate, arrCanCharge, fltDt, fltBatteryCap)
constraints, obj_value = make_objectives(constraints, varCharge, varDischarge, varSocs, \
                        arrCanCharge, arrChargeRate,arrSOCconstraint,fltBatteryCap,fltDt, \
                                         fltRegDownVal, fltRegUpVal,lsElectric)
    

obj = cvx.Minimize(obj_value)
prob = cvx.Problem(obj, constraints)

prob.solve(verbose=True)

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 10080, constraints m = 28203
          nnz(P) + nnz(A) = 38220
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.2258e+05   1.55e+02   1.44e+02   1.00e-01   1.30e-02s
 200  -2.6380e+03   3.83e-01   1.01e+00   1.00e-01   9.39e-02s
 400  -4.3640e+03   1.26e+00   2.96e-01

SolverError: Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.